# This notebook adds the necessary columns to the dataset

In [1]:
import pandas as pd
import numpy as np
import string
import seaborn as sns
from matplotlib import pyplot as plt
from cycler import cycler
import plotly.graph_objects as go
import re
from matplotlib.lines import Line2D

In [2]:
data = pd.read_csv("GM_new_types_3583.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3593 entries, 0 to 3592
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bca_id           3593 non-null   object 
 1   building         3593 non-null   object 
 2   award            3593 non-null   object 
 3   fy               3593 non-null   object 
 4   gfa              3593 non-null   object 
 5   address          3591 non-null   object 
 6   postal_code      3581 non-null   object 
 7   district         3593 non-null   object 
 8   developer        3264 non-null   object 
 9   architect        1379 non-null   object 
 10  structural_eng   1452 non-null   object 
 11  me_eng           1470 non-null   object 
 12  land_cons        1195 non-null   object 
 13  qs               0 non-null      float64
 14  main_contractor  1264 non-null   object 
 15  fac_mgt          1908 non-null   object 
 16  esd_cons         3591 non-null   object 
 17  esco_cons     

In [3]:
data.columns

Index(['bca_id', 'building', 'award', 'fy', 'gfa', 'address', 'postal_code',
       'district', 'developer', 'architect', 'structural_eng', 'me_eng',
       'land_cons', 'qs', 'main_contractor', 'fac_mgt', 'esd_cons',
       'esco_cons', 'green_cons', 'description', 'green_features',
       'building_use'],
      dtype='object')

3593 entries from the original csv file (before removing anything)

In [4]:
data = data[data["green_features"] != "[]"] # 2 entries with no green features initially
data = data.fillna({"address": "Singapore"}) #entries with empty address fields are in Singapore

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3591 entries, 0 to 3592
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bca_id           3591 non-null   object 
 1   building         3591 non-null   object 
 2   award            3591 non-null   object 
 3   fy               3591 non-null   object 
 4   gfa              3591 non-null   object 
 5   address          3591 non-null   object 
 6   postal_code      3579 non-null   object 
 7   district         3591 non-null   object 
 8   developer        3264 non-null   object 
 9   architect        1377 non-null   object 
 10  structural_eng   1451 non-null   object 
 11  me_eng           1468 non-null   object 
 12  land_cons        1193 non-null   object 
 13  qs               0 non-null      float64
 14  main_contractor  1262 non-null   object 
 15  fac_mgt          1906 non-null   object 
 16  esd_cons         3589 non-null   object 
 17  esco_cons     

3591 entries after removing 2 entries with no green features

In [6]:
def remove_suffix(bdg_name):
    new_name = bdg_name
    """
    removes the 're-certified' and related suffixes
    this is for later when finding change in ratings and new vs recertified
    """
    suffixes_to_remove = [" (1st recertification)", " (1st Recertification)", " (1st Re certification)", " (recertification)",\
                          " (Recertification)", " (2nd recertification)", " (Re certified)", " (3rd recertification)",\
                          " (Re certification)", " (2nd Re certification)", " (3rd Re certification)",\
                          " (1st Re certificataion)", " (1st Re certifcation)", " (Re Certification)"]
    for suffix in suffixes_to_remove:
        if suffix in bdg_name:
            new_name = bdg_name.replace(suffix, "")
    return new_name

In [7]:
data["building"] = data["building"].map(remove_suffix)

# Column for building use (manual)

In [8]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
def remove_punctuation(word):
    punc = string.punctuation
    for p in punc:
        word = word.replace(p,"")
    return word

In [10]:
def contains_keyword(desc, keywords):
    """
    check if the description of the building contains any keyword from a list of keywords
    """
    for word in keywords:
        if word in desc:
            return True
    return False

In [11]:
residential_keywords = ["residence", "residences", "residential", "condominium", "terrace", "accommodation", "hdb", "housing",\
                       "hostel", "apartment", "apartments", "bedroom", "welfare", "dwelling", "landed", "hospice", "homes",\
                       "rivergate"]

dorm_keywords = ["dorm", "lodge"]

rec_culture_keywords = ["recreation", "recreational", "entertainment", "reserve", "garden",\
                        "exhibition", "sports", "club", "theatre", "museum", "community", "marshes", "library",\
                        "temple", "stables", "wetlands", "cultural", "hortpark", "cinema", "sportshub", "sentosa",\
                       'clubhouse']

office_keywords = ["office", "business", "kpmg", "manulife", "insurance", "db&b", "management", "registry", "langdon",\
                  "offices", "merrill", "mediacorp", "autodesk", "bosch", "hitachi", "bnp", "german", "71", "biopolis",\
                  "headquarter", "fusionopolis", "starhub", "challenger", "ramdas"]

commercial_keywords = ["commercial", "services", "shopping", "retail", "plaza", "market", "restaurant", "food", "supermarket",\
                      "market/hawker", "hawker", "grand shanghai", "retails", "downtown", "cold storage", "jth", "spa",\
                      "shopping/entertainment", "complex", "daiso", "dining", "mice", "elo", "shophouses", "gravity",\
                      "mcdonalds", "mall", "posb", "ocbc", "dbs", 'northpoint', 'palais', 'cafe', 'cafÃ©',\
                      'office-cum-retail'] + office_keywords

hotel_keywords = ["hotel", "villa", "villas", "parkroyal"]

industrial_keywords = ["industrial", "workshop", "plant", "factory", "warehouse", "motorcycles", "transicold", "abbvie",\
                      "sunray", "surbana", "jtc", "keppel", "motors", "samwoh", "kerry", "micron"]

hospital_keywords = ["hospital", "polyclinic", "polyclinics", "ncid"]

edu_research_keywords = ["education", "institute", "institution", "school", "college", "childcare", "university",\
                        "laboratory", "r&d", "polytechnic", "laboratories", "classrooms", "preschool", "nus", "bca", "sim",\
                        "liveable", "educational", "pharmaceuticals", "research", "institutional", "innovillage"]

govt_security_keywords = ["parliament", "ministry", "police", "prison", "barrage", "training", "ns16", "1337", "depot"]

data_centre_keywords = ["data"]

airport_keywords = ["airport", "terminal"]

mixed_keywords = ["mixed", "tripleone", "goodman", "jcube"]

In [12]:
bdg_use_names = ["residential", "dormitory", "recreational/culture", "office/commercial", "hotel", "industrial",\
                "hospital/clinic", "educational/research", "government/security", "data centre", "airport", "mixed"]

In [13]:
def contains_phrase(keyphrase, desc):
    for i in keyphrase.split():
        if i not in desc:
            return False
    return True

In [14]:
"""
adding column for building use using the description and name
"""

data["building_use_manual"] = None


for i in data.index:
    building_use_manual = []
    desc = list(map(remove_punctuation, data.at[i, "description"].lower().split()))
    desc2 = list(map(remove_punctuation, data.at[i, "building"].lower().split()))
    desc.extend(desc2)
    # split to avoid including words like "carpark" into recreational buildings
    # remove punctuations to avoid keywords with punctuation not matching (eg 'university.')
    
    if contains_keyword(desc, residential_keywords) or ("block" in desc and "658" in desc) or\
    contains_phrase("b house", desc) or contains_phrase("detached house", desc):
        building_use_manual.append("residential")
    
    if contains_keyword(desc, dorm_keywords):
        building_use_manual.append("dormitory")
    
    if contains_keyword(desc, rec_culture_keywords) or contains_phrase("regional park", desc) or\
    ("park" in desc and "car" not in desc):
        building_use_manual.append("recreational/culture")
    
#     if contains_keyword(desc, office_keywords)\
#     or contains_phrase("financial centre", desc) or contains_phrase("convention centre", desc) or\
#     contains_phrase("asia square", desc) or contains_phrase("city house", desc) or \
#     contains_phrase("standard chartered bank", desc):
#         building_use_manual.append("office")
    
    if contains_keyword(desc, commercial_keywords) or contains_phrase("cold storage", desc) or\
    contains_phrase("jurong port", desc) or contains_phrase("xtra hypermarket", desc) or\
    contains_phrase("financial centre", desc) or contains_phrase("convention centre", desc) or\
    contains_phrase("asia square", desc) or contains_phrase("city house", desc) or \
    contains_phrase("standard chartered bank", desc):
        building_use_manual.append("office/commercial")
    
    if contains_keyword(desc, hotel_keywords) or contains_phrase("millennium and copthorne", desc):
        building_use_manual.append("hotel")
    
    if contains_keyword(desc, industrial_keywords) or contains_phrase("rolls royce", desc) or \
    contains_phrase("agc group", desc):
        building_use_manual.append("industrial")
    
    if contains_keyword(desc, hospital_keywords):
        building_use_manual.append("hospital/clinic")
    
    if contains_keyword(desc, edu_research_keywords) or contains_phrase("enrichment centre", desc) or\
    contains_phrase("my tree house", desc) or contains_phrase("nanyang auditorium", desc) or\
    contains_phrase('nanyang house', desc):
        building_use_manual.append("educational/research")
    
    if contains_keyword(desc, govt_security_keywords) or contains_phrase("fire station", desc) or\
    contains_phrase("detention tank", desc) or contains_phrase("supreme court", desc) or \
    contains_phrase("circle line", desc) or contains_phrase("mce ventilation", desc) or\
    contains_phrase("detention barrack", desc) or contains_phrase("depot extension", desc)\
    or contains_phrase("changi naval base", desc) or contains_phrase("upper thomson station", desc) or \
    contains_phrase("singapore tourism board", desc) or contains_phrase("sungei gedong camp", desc)\
    or contains_phrase("singapore telecommunications", desc) or contains_phrase("boys' home", desc) or\
    contains_phrase("government technology agency", desc) or contains_phrase("bus interchange", desc):
        building_use_manual.append("government/security")
    
    if contains_keyword(desc, data_centre_keywords):
        building_use_manual.append("data centre")
        
    if contains_keyword(desc, airport_keywords):
        building_use_manual.append("airport")
        
    if contains_keyword(desc, mixed_keywords):
        building_use_manual.append("mixed")

    data.at[i, "building_use_manual"] = building_use_manual

In [15]:
data.to_csv("greenmark_bdgs_added_cols.csv", index=False)

In [16]:
len(data[data["building_use_manual"].str.len() == 0])

552

In [17]:
def most_frequent_word(list_of_sentences):
    d = {}
    for sentence in list_of_sentences:
        for word in sentence.split():
            if word not in d:
                d[word] = 0
            d[word] += 1
    return dict(reversed(sorted(d.items(), key=lambda x: x[1])))

In [18]:
most_frequent_word(list(data[data["building_use_manual"].str.len() == 0]["description"]))

{'-': 516,
 'of': 28,
 'a': 21,
 'at': 20,
 'and': 20,
 'is': 18,
 'building': 15,
 'the': 12,
 'to': 12,
 'with': 11,
 'located': 11,
 'The': 9,
 '6': 6,
 'in': 6,
 'floor': 5,
 'has': 5,
 'tenanted': 5,
 '1': 5,
 'AVENUE': 5,
 'AT': 5,
 'ON': 5,
 'OF': 5,
 'PROPOSED': 5,
 'PT,': 4,
 'area': 4,
 'total': 4,
 'storey': 4,
 'multi': 4,
 'Road.': 3,
 'mix': 3,
 'instant': 3,
 'coffee': 3,
 'Group': 3,
 'GFA': 3,
 'It': 3,
 'MRT': 3,
 'AREA)': 3,
 'PLANNING': 3,
 '3': 3,
 '&': 3,
 'centre': 3,
 'basement': 3,
 '2': 3,
 'A': 3,
 'ERECTION': 3,
 'facility': 3,
 'Hub': 3,
 'LOT': 3,
 'STATION': 3,
 'Pit': 2,
 'F1': 2,
 '80': 2,
 '1997': 2,
 'completed': 2,
 'was': 2,
 'Havelock': 2,
 '390': 2,
 'office-cum-retail': 2,
 '8': 2,
 '11': 2,
 'WOODLANDS': 2,
 '1Location': 2,
 'storey:': 2,
 'm2No': 2,
 'RestaurantSize:': 2,
 'Use:': 2,
 'Type': 2,
 'cafÃ©': 2,
 'engaged': 2,
 'Lot': 2,
 'on': 2,
 'no.': 2,
 'Proposed': 2,
 'ROAD': 2,
 'BEDOK': 2,
 'Place': 2,
 'station.': 2,
 'Centre': 2,
 'Marin

In [19]:
no_use = data[data["building_use_manual"].str.len() == 0]
no_use

,bca_id,building,award,fy,gfa,address,postal_code,district,developer,architect,...,qs,main_contractor,fac_mgt,esd_cons,esco_cons,green_cons,description,green_features,building_use,building_use_manual
0,799B4F1B-A16E-4F38-84A2-F579864E95E9,orchardgateway@emerald,Green Mark Gold,FY18,9733,"218 Orchard Road, Singapore 238851",238851,9,218 Orchard Private Limited,NaN,...,NaN,NaN,NaN,-,-,-,-,['-'],Commercial,[]
38,28D5033E-AF2C-4C7E-91A8-414A705682A5,Caltex Jurong Spring,Green Mark Platinum (Super Low Energy),FY20,812,100 Jurong West Avenue 1,649519,22,Chevron Singapore Pte Ltd,OGP Konsult Pte Ltd,...,NaN,Wing Tuck Engineering Pte Ltd,NaN,-,-,-,PROPOSED RECONSTRUCTION OF EXISTING SINGLE STO...,"['5 Ticks Unitary Air-conditioning System', '1...",Others,[]
54,93039df1-5422-4753-b9d5-b93a47951566,Mapletree Benoi Logistic Hub,Green Mark Platinum (Super Low Energy),FY21,92463,21 Benoi Sector,629853,22,NaN,NaN,...,NaN,NaN,Mapletree Property Management Pte Ltd,Building System and Diagnostics Pte Ltd,-,-,Mapletree Benoi Logistics Hub is a five-storey...,['Extensive use of recycled demolished materia...,Industrial,[]
60,5FB8C5EE-09F0-48EB-8AB1-7EDCA52DB8A2,Anadarko Singapore Pte Ltd,Green Mark Gold,FY18,317,"138 Market Street, #15-03 CapitaGreen, Singapo...",048946,1,Anadarko Singapore Pte Ltd,NaN,...,NaN,NaN,NaN,-,-,-,-,['-'],Office Interior,[]
75,0E674118-37F2-4F45-BE15-1D6881977499,ST Engineering Electronics,Green Mark Gold,FY20,20931,"100 Jurong East Street 21, Singapore 609602",-,22,ST Engineering Electronics Ltd,Kyoob Architects Pte Ltd,...,NaN,NaN,NaN,-,-,-,-,['-'],Industrial,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,1DE1FF1C-0FFF-4FDE-96CD-3B2A30AE9F80,CPG Consultants Pte Ltd,Green Mark Gold Plus,FY19,8000,"1 Gateway Drive, #25-01 Westgate Tower, Singap...",-,22,CPG CONSULTANTS PTE LTD,NaN,...,NaN,NaN,NaN,-,-,-,-,['-'],Office Interior,[]
3529,A8B8DC7C-CEB4-420B-A40C-E6FFED2A074D,China International Capital Corporation (Singa...,Green Mark Certified,FY18,430,"6 Battery Road, #33-01, Singapore 049909",049909,1,China International Capital Corporation (Singa...,NaN,...,NaN,NaN,NaN,-,-,-,-,['-'],Office Interior,[]
3543,6CD7A905-5A25-4EE6-AFC5-36DB82FC6C0C,Tokyo Gas Asia Pte Ltd,Green Mark Certified,FY19,298.6,"9 Raffles Place, #22-03 Republic Plaza, Singap...",48619,1,Tokyo Gas Asia Pte Ltd,NaN,...,NaN,NaN,NaN,-,-,-,-,['-'],Office,[]
3547,9C6B5990-6412-4771-920B-3EEB3468D4C7,Chinatown Point,Green Mark Certified,FY19,53610.72,"133 New Bridge Road, Singapore 059413",059413,1,MCST 1789,NaN,...,NaN,NaN,NaN,-,-,-,-,['-'],Retail,[]


In [20]:
for _,row in no_use.sample(10)[["building", "description"]].iterrows():
    print([row["building"], row["description"]])

['Singapore Land Authority, GeoWorks', '-']
['Alkaff Oasis (Contract 7)', '-']
['Wheelock Place', '-']
['Singapore Water Exchange, PUB WaterHub', '-']
['Solaris Building', '-']
['7 North Coast', '-']
['Just Dough', '-']
['Millenia Tower', '-']
['Tanah Merah 1', '-']
['Admiralty Flora', '-']


In [21]:
descs = list(data[data["building_use_manual"].str.len() == 0]["description"])
#description of buildings still with no building use input

In [22]:
names = list(data[data["building_use_manual"].str.len() == 0]["building"])
#names of buildings still with no building use input

In [23]:
def combine_lsts(lst1, lst2):
    new = []
    for i in range(len(lst1)):
        tup = (lst1[i], lst2[i])
        new.append(tup)
    return new

# Removing purely residential buildings

In [24]:
len(data[data["building_use"] == 'Residential']) # 410 with type "Residential"

410

In [25]:
len(data[data["building_use"] == 'Public Housing']) # 73 with type "Public Housing"

73

In [26]:
data = data.drop(data[data["building_use"] == "Residential"].index)

In [27]:
data = data.drop(data[data["building_use"] == "Public Housing"].index)

In [28]:
# data["building_use_manual"] = data["building_use_manual"].astype(str)

In [29]:
# len(data[data["building_use_manual"] == "['residential']"])

In [30]:
# data = data[data["building_use_manual"] != "['residential']"]

In [31]:
# data["building_use_manual"] = data["building_use_manual"].apply(eval) # turn the string values back to list

In [32]:
len(data)

3108

### Helper function

In [33]:
def flatten(lst):
    new = []
    for i in lst:
        for j in i:
            new.append(j)
    return new

def count_vals(lst):
    """
    creates a dic where {unique_value: count} for a list
    """
    new = {}
    for i in lst:
        if i not in new:
            new[i] = 1
        else:
            new[i] += 1
    return new

## Drop entries with no award data

In [34]:
data[data['award'] == "-"] # 2 entries with no award data

,bca_id,building,award,fy,gfa,address,postal_code,district,developer,architect,...,qs,main_contractor,fac_mgt,esd_cons,esco_cons,green_cons,description,green_features,building_use,building_use_manual
591,4efb2b99-d482-4668-b074-6d014829070a,NUH Medical Centre,-,FY21,71917,1 Lower Kent Ridge Road,119082,5,National University Health System,NaN,...,NaN,NaN,Operations Support Services,G-Energy Global Pte Ltd,-,-,The 19-storey centre caters mainly to patients...,['Energy efficient chiller plant with a system...,Office,[]
3319,f1c2f46b-dfc9-41da-9970-88c6f8866940,Thomson Medical Centre,-,FY21,12495.85,"339 Thomson Road, Singapore 307677",307677,11,Thomson Medical Centre,NaN,...,NaN,NaN,NaN,SAC Energy (S) Private Limited,SAC Energy (S) Private Limited,-,"Established in 1979, Thomson Medical is Singap...",['Use of efficient Water-Cooled Chilled -Water...,Office,[office/commercial]


In [35]:
data = data.drop(data[data["award"] == "-"].index)

In [36]:
len(data)

3106

## Drop entries with no green features

In [37]:
len(data[data["green_features"] == "['-']"]) # 1383 further entries with no green features

1383

In [38]:
data = data.drop(data[data["green_features"] == "['-']"].index)

In [39]:
len(data)

1723

# Adding column for main category

In [40]:
keywords_df = pd.read_csv("keywords_new.csv")
keywords_df

,categories,subcategories,keywords
0,sustainable management,empty,"[""green mark manager"", ""energy manager"", ""gree..."
1,building energy performance,empty,"[""ettv"", ""envelope thermal transfer value"", ""i..."
2,resource stewardship,empty,"[""wels"", ""water efficiency labelling scheme"", ..."
3,smart & healthy building,empty,"[""occupant comfort"", ""thermal comfort"", ""tempe..."
4,advanced green effort,empty,"[""green mark pearl"", ""occupant-centric"", ""occu..."


In [41]:
# Adding columns for category and subcategory
data['main_category'] = None
# data['subcategory'] = None
for _,row in data.iterrows():
    mc = []
#     sc = []
    gfs = row["green_features"]
    gfs = eval(gfs)
    for gf in gfs:
        gf = gf.lower()
        for __,rowg in keywords_df.iterrows():
            kws = rowg["keywords"]
            kws = eval(kws)
            for k in kws:
                if k in gf:
                    mc.append(rowg["categories"])
#                     sc.append(rowg["subcategories"])
    data.at[_, "main_category"] = mc
#     data.at[_, "subcategory"] = sc

## Adding column for main category (grouped)
main category (grouped) looks at each buildings' green features

ungrouped looks at each green feature as a whole

In [42]:
data["main_category_grouped"] = None
for i in data.index:
    data.at[i, "main_category_grouped"] = list(set(data.at[i, "main_category"]))

In [43]:
data.to_csv("GM_added_cols.csv", index=False)

In [44]:
len(data)

1723

## Remove buildings with no given building type

In [45]:
data = data.dropna(subset=["building_use"]) # 137 bdgs with no given building type

In [46]:
len(data)

1586

## Group building types

In [47]:
sorted(data["building_use"].unique())

['Commercial',
 'Data Centre',
 'District',
 'Healthier Workplaces',
 'Hotel',
 'Industrial',
 'Infrastructure',
 'Institutional',
 'Institutional (Healthcare)',
 'Institutional (School)',
 'Laboratories',
 'Mixed Development',
 'Office',
 'Office Interior',
 'Others',
 'Park',
 'Restaurant',
 'Retail',
 'Retail (Tenant)',
 'Supermarkets']

In [48]:
data.replace(["Office", "Office Interior", "Healthier Workplaces"], "Commercial", inplace=True)
data.replace(["Data Centre"], "Industrial", inplace=True)
data.replace(["Laboratories", "Institutional (School)", "Institutional (Healthcare)"], "Institutional", inplace=True)
data.replace(["Restaurant", "Supermarkets", "Retail (Tenant)"], "Retail", inplace=True)
data.replace(["District", "Infrastructure", "Mixed Development", "Park"], "Others", inplace=True)

In [49]:
sorted(data["building_use"].unique())

['Commercial', 'Hotel', 'Industrial', 'Institutional', 'Others', 'Retail']

In [50]:
'''
Commercial - office, office interior, healthier workplace
Industrial - data centre
Institutional - lab, 
Hotel
Retail - restaurant, supermarkets
Others - district, infrastructure, mixed dev, park 
'''

'\nCommercial - office, office interior, healthier workplace\nIndustrial - data centre\nInstitutional - lab, \nHotel\nRetail - restaurant, supermarkets\nOthers - district, infrastructure, mixed dev, park \n'

In [51]:
data.to_csv("GM_given_types_cleaned_3583.csv", index=False)

In [52]:
len(data)

1586

## 1586 entries

Interestingly, there are FY22 buildings but all of them do not have green features, so they are all removed from the dataset